# E1:统计土耳其公民中所有人中年龄最大的男人
## 方法：用map-reduce的方法处理RDD
## 备注：自己电脑内存小，分批处理然后手动汇总

In [1]:
# -*- coding: utf-8 -*-
from pyspark import SparkContext, SparkConf
filePath = './mernis/data_dump.sql'
batchSize = 10000
columns = ['uid', 'national_identifier', 'first_name', "last_name",'mother_first','father_first', 'gender', 'birth_city',
'date_of_birth', 'id_registration_city', 'id_registration_district', 'address_city', 'address_district',
'address_neighborhood', 'street_address', 'door_or_entrance_number', 'misc']
appName = 'pj2'

conf = SparkConf().setAppName(appName)
sc = SparkContext(conf=conf)

In [2]:
def mapMaxAgeMan(line):
    age = 2019 - int(line[1][-4:])
    return line[0],age
def reduceMaxAgeMan(a,b):
    return max(a,b)

In [3]:
local_maxs = []
with open(filePath, 'r') as f:
    for i in range(75):
        f.readline()
    #print(f.readline().strip().split('\t'))
    for i in range(int(12000000/batchSize)):
        batchData = []
        for j in range(batchSize):
            line = f.readline().strip().split('\t')
            if line[6]=='E':
                line = [line[1],line[8]]
                batchData.append(line)
        #batchData=pd.DataFrame(batchData,columns=columns)
        rdd = sc.parallelize(batchData)
        mapE1 = rdd.map(mapMaxAgeMan)
        # man_age = mapE1.reduceByKey(max).collect()
        local_maxs.append(mapE1.max(lambda x:x[1])) # 找到本组内最大的
        # local_maxs.append(man_age)
        rdd.unpersist() # 缓解内存压力
        mapE1.unpersist()

In [4]:
local_maxs # 120个

[('61375077714', 107),
 ('31085095292', 105),
 ('30986091310', 116),
 ('58492175270', 118),
 ('10979410236', 103),
 ('46903568124', 101),
 ('25207935542', 108),
 ('62449042898', 102),
 ('43576661794', 104),
 ('60892096246', 120),
 ('30491108466', 110),
 ('54994291036', 109),
 ('27076608770', 103),
 ('44401641318', 115),
 ('39793184734', 105),
 ('55840263788', 105),
 ('37384529988', 103),
 ('25466262946', 108),
 ('12868387384', 109),
 ('43372683938', 107),
 ('31724066638', 116),
 ('56035257520', 107),
 ('21380412764', 115),
 ('26075248048', 103),
 ('10577772668', 103),
 ('23033357804', 102),
 ('46576223702', 114),
 ('42049717846', 117),
 ('46768217304', 104),
 ('27317215132', 686),
 ('59842131250', 114),
 ('61828065084', 118),
 ('62530047030', 120),
 ('59476225448', 112),
 ('47071557040', 109),
 ('10136438332', 120),
 ('39976875688', 117),
 ('59158237066', 101),
 ('50092539384', 105),
 ('45367696916', 105),
 ('49192570816', 105),
 ('37372966924', 117),
 ('65920013578', 102),
 ('20684521

In [5]:
sc.parallelize(local_maxs).max(lambda x:x[1]) #调出全局最大

('27317215132', 686)

In [ ]:
local_maxs = []
with open(filePath, 'r') as f:
    for i in range(75):
        f.readline()
    #print(f.readline().strip().split('\t'))
    for i in range(int(12000000/batchSize)):
        batchData = []
        for j in range(batchSize):
            line = f.readline().strip().split('\t')
            if line[6]=='E':
                line = [line[1],line[8]]
                batchData.append(line)
        #batchData=pd.DataFrame(batchData,columns=columns)
        rdd = sc.parallelize(batchData)
        mapE1 = rdd.map(mapMaxAgeMan)
        # man_age = mapE1.reduceByKey(max).collect()
        local_maxs.append(mapE1.max(lambda x:x[1] )) # 找到本组内最大的
        # local_maxs.append(man_age)
        rdd.unpersist() # 缓解内存压力
        mapE1.unpersist()

# E2. 统计姓名中最常出现的字母
## 方法：换一种思路实现，尝试使用Spark SQL
## 备注：因为听说spark sql 很方便，而且对数据结构做了优化，dataframe和dataset比rdd快了不少。这个task在1309做，可以尝试一次性把所有内容加载进内存

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext